In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], "src"))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [5]:
from pkg.geometry.geometry import *

In [6]:
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (0,0.2,0), 
                           rpy=(np.pi/2,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.2,-0.5,0.),rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (-0.2,-0.5,0), rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (0.1,0.5,0.01), (0,-0.5,0), rpy=(0,0,np.pi/2), 
                           color=(1,0.7,0.7,0.5), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (-0.4,-0.3,0), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [7]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add brush

In [8]:
brush_body = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", 
                                dims=(0.2,0.07,0.02), center=(0.0,-0.5,0.045), rpy=(0,0,np.pi/2), 
                                color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
brush_handle = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", 
                                  dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                                  color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
brush_face = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", 
                                dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                                color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
sweep = pscene.create_object(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

## Set test scenario

In [16]:
from pkg.planning.constraint.constraint_common import combine_redundancy, sample_redundancy
import random

actor = pscene.actor_dict["grip0"]
obj = pscene.subject_dict["brush"]
handle = obj.action_points_dict["handle"]
redundancy_tot = combine_redundancy(handle, actor)
redundancy = sample_redundancy(redundancy_tot, sampler=random.uniform)
redundancy['grip0']['w'] = np.deg2rad(0)
redundancy['handle']['w'] = np.deg2rad(0)
Q_dict = crob.home_dict

## Grasp checker

In [17]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene, end_link_couple_dict= {"indy0_tcp": ["indy0_tcp", "indy0_link6"], "base_link":["base_link"]})

## grasp check

In [18]:
gtimer = GlobalTimer.instance()
gtimer.reset()

for _ in range(1000):    
    with gtimer.block("check"):
        res = gcheck.check(actor, obj, handle, redundancy, Q_dict)
print(res)
print(gtimer)

True
check: 	361.0 ms/1000 = 0.361 ms (0.344/0.753)



## Reach SVM

In [19]:
from pkg.planning.filtering.reach_filter import ReachChecker
rcheck = ReachChecker(pscene)

## reach check

In [20]:
gtimer = GlobalTimer.instance()
gtimer.reset()

for _ in range(1000):    
    with gtimer.block("check"):
        res = rcheck.check(actor, obj, handle, redundancy, Q_dict)
print(res)
print(gtimer)

True
check: 	198.0 ms/1000 = 0.198 ms (0.188/0.532)



## Latticized Filter

In [21]:
from pkg.planning.filtering.latticized_filter import LatticedChecker
lcheck = LatticedChecker(pscene, end_link_couple_dict= {"indy0_tcp": ["indy0_tcp", "indy0_link6"], "base_link":["base_link"]})

## latticize and check

In [22]:
gtimer = GlobalTimer.instance()
gtimer.reset()

for _ in range(10):    
    with gtimer.block("check"):
        res = lcheck.check(actor, obj, handle, redundancy, Q_dict)
print(res)
print(gtimer)

True
check: 	339.0 ms/10 = 33.868 ms (30.631/44.847)



## actual plan

In [23]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

In [24]:
initial_state = pscene.update_state(crob.home_pose)
grasp_state = initial_state.copy(pscene)
grasp_state.set_binding_state((('brush', 'handle', 'grip0', 'grip0'), ('sweep', None, None, None)), pscene)
redundancy_dict = {"brush": redundancy}

In [25]:
Traj, LastQ, error, success, binding_list = mplan.plan_transition(initial_state, grasp_state, 
                                                                  redundancy_dict=redundancy_dict, timeout=5)
print(success)

True


In [26]:
gscene.show_motion(Traj)

In [27]:
gscene.show_pose(Traj[-1])

### planning pipeline

In [28]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, [gcheck, rcheck, lcheck])
mplan.update_gscene()
from pkg.planning.task.object_a_star import ObjectAstar
tplan = ObjectAstar(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## ui

In [29]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


## Test plan

In [30]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.update_state(crob.home_pose)

   Use a production WSGI server instead.


In [31]:
mplan.reset_log(True)
# goal_nodes = [("goal",)]
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=True, dt_vis=0.01, timeout_loop=300, multiprocess=False, timeout=1)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

 * Debug mode: off
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('grip0', 0) = success
Remaining:4->3 / branching: 0->1 (0.27/300.0 s, steps/err: 9(257.592201233 ms)/0.0010178227688)
node: ('grip0', 0)->('grip0', 1) = success
Remaining:3->2 / branching: 1->2 (0.39/300.0 s, steps/err: 3(118.279933929 ms)/0.00122253147001)
node: ('grip0', 1)->('grip0', 2) = success
Remaining:2->1 / branching: 2->3 (2.97/300.0 s, steps/err: 118(2571.93303108 ms)/0.00180571049773)
node: ('grip0', 2)->('goal', 2) = success
Remaining:1->0 / branching: 3->4 (3.17/300.0 s, steps/err: 7(199.262142181 ms)/0.00185523603999)
======================= terminated 0: first answer acquired ===============================


In [32]:
ppline.play_schedule(snode_schedule, period=0.01)

In [33]:
print(gtimer)

test_transition: 	3148.0 ms/7 = 449.768 ms (0.164/2571.933)
GraspChecker: 	3.0 ms/5 = 0.507 ms (0.402/0.692)
ReachChecker: 	1.0 ms/4 = 0.289 ms (0.245/0.417)
LatticedChecker: 	138.0 ms/4 = 34.601 ms (32.107/40.53)
planning: 	162.0 ms/4 = 40.418 ms (15.651/95.201)



In [34]:
mplan.result_log

{'GraspChecker': [False, True, True, True, True],
 'LatticedChecker': [True, True, True, True],
 'ReachChecker': [True, True, True, True],
 'planning': [True, True, True, True]}

## Text execution

In [21]:
# crob.reset_connection([True])

connection_list
[True]
Connect: Server IP (192.168.0.63)


## unsynced execution

In [23]:
ppline.execute_schedule(snode_schedule, vel_scale=0.5)

binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: brush_face
rname: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: brush_face
rname: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: goal_bd
rname: None
binder: brush_face
rname: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
go
Connect: Server 

## synced execution

In [ ]:
## 
# with RvizPublisher(gscene, []) as rpub:
#     ppline.execute_schedule(snode_schedule, rviz_pub=rpub)

## Disconnect stereo

In [ ]:
stereo.disconnnect()